In [1]:
import sys

sys.path.insert(0,'/regelum-playground')

import numpy as np

from src.simulator_gz import Robot3Pi
from src.system_exp import ThreeWheelPushingObject
from src.environment import EnvironmentConfig, Task, TwistAction, PushingObject, RobotPursuit

from src.simulator_gz import Robot3PiLineFollowing
from src.system_exp import ThreeWheelLineFollowing
from src.environment import LineFollowing

from src.simulator_gz import Robot3PiRobotPursuit
from src.system_exp import ThreeWheelPushingObject
from src.environment import RobotPursuit

# Pushing Objects

In [2]:
## Possible Tasks (With either 15 or -15 rotation as starting point)
tasks = {}
tasks['red'] = Task('red',[Task.Transform(position=[0.4,0.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,0.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=20)
tasks['green'] = Task('green',[Task.Transform(position=[0.4,4.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,4.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=20)
tasks['blue'] = Task('blue',[Task.Transform(position=[0.4,8.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,8.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=0)
tasks['yellow'] = Task('yellow',[Task.Transform(position=[0.4,-4.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,-4.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=0)
tasks['pink'] = Task('pink',[Task.Transform(position=[0.4,-8.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,-8.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=0)
tasks['cyan'] = Task('cyan',[Task.Transform(position=[0.4,-12.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,-12.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=20)


actions = []

raw_action_space = 4.*np.array([[0.,0.],[0.5,0.1],[0.1,0.5],[0.3,0.3]]) ;

for e in raw_action_space:
    label = "" ;
    if e[0]+e[1] < 0.0001: label = "stop" ;
    elif e[0] == e[1]: label = "straight" ;
    elif e[0] >= e[1]: label = "right" ;
    elif e[0] <= e[1]: label = "left" ;
    actions.append(TwistAction(label, e))

In [3]:
system = ThreeWheelPushingObject()
env_config = EnvironmentConfig(observation_shape=[20,20,3],
                               tasks=tasks,
                               actions=actions,
                               robot_name='3pi_front_cam_robot',
                               vehicle_prefix='/vehicle',
                               world_name='/world/pushing_objects_world',
                               camera_topic='/vehicle/camera')

In [4]:
sim_rob = Robot3Pi(system, 
                   state_init=np.zeros([20*20*3]),
                   time_final=30)

In [5]:
from src.objective import ThreeWheeledRobotCostWithSpot, PushingObjectRunningObjective
from regelum.model import ModelQuadLin


objective_function = ThreeWheeledRobotCostWithSpot(
    quadratic_model=ModelQuadLin(
        weights= [100, 100, 0, 0, 0],
        quad_matrix_type="diagonal",
        is_with_linear_terms=False),
        spot_gain=500,
        spot_x_center=-1.5,
        spot_y_center=-1.5,
        spot_std=0.2,
    )

objective_function = PushingObjectRunningObjective()

In [6]:
po = PushingObject(sim_rob, 
                   running_objective=objective_function)

simulator: <src.simulator_gz.Robot3Pi object at 0x71980c181ab0>
subscribed to Camera!
Subscribed to dynamic_pose/info!
Waiting for simulator... 2.82 sec

KeyboardInterrupt: 

In [ ]:
po.switch_task(0)
observation, _ = po.reset()

In [ ]:
import time

observation, _ = po.reset()

for _ in range(10):
    observation, reward, truncated, terminated, info = po.step(np.array([1, 0]))
    if np.logical_or(truncated, terminated):
        break
    
    time.sleep(0.2) 

po.step(np.array([0, 0]))

(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.42352942 0.85490197 16
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.42352942 0.85490197 20
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.42352942 0.85490197 30
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.38039216 0.85490197 28
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.6666667 V 0.3529412 0.85490197 25
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.0 V 0.42352942 0.85490197 0
COND: LOST
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.0 V 0.42352942 0.85490197 0
COND: LOST
self.system.inputs: [[0. 0.]]


(
    array([[0.85490197, 0.85490197, 0.85490197, ..., 0.42352942, 0.42352942,
        0.42352942]], dtype=float32),
    1.0,
    True,
    False,
    {}
)

# Line Following

In [ ]:
system = ThreeWheelLineFollowing()
sim_rob = Robot3PiLineFollowing(system, 
                                state_init=np.zeros([2*50*3]),
                                time_final=30)

In [ ]:
from src.objective import LineFollowingRunningObjective


objective_function = LineFollowingRunningObjective()
lf = LineFollowing(sim_rob, 
                   running_objective=objective_function)

simulator: <src.simulator_gz.Robot3PiLineFollowing object at 0x7bbd3ee57940>
subscribed to Camera!
Subscribed to dynamic_pose/info!
Waiting for simulator... 0.22 sec
Scene received!
Model in scene colored_ground_plane
Model in scene 3pi_robot
Model in scene top_camera

Observation and position received!


In [ ]:
lf.switch_task(0)
observation, _ = lf.reset()

In [ ]:
import time

observation, _ = lf.reset()

for _ in range(10):
    observation, reward, truncated, terminated, info = lf.step(np.array([1, 0]))
    print(f"truncated: {truncated}, terminated: {terminated}")
    if np.logical_or(truncated, terminated):
        break
    
    time.sleep(0.2) 

lf.step(np.array([0, 0]))

truncated: False, terminated: False
Counter overflow with t0: 3.565 and last moment: 3.697
truncated: False, terminated: False
truncated: False, terminated: False
Counter overflow with t0: 3.862 and last moment: 3.9939999999999998
truncated: False, terminated: False
Counter overflow with t0: 4.027 and last moment: 4.159
truncated: False, terminated: False
Counter overflow with t0: 4.192 and last moment: 4.324
truncated: False, terminated: False
truncated: False, terminated: False
Counter overflow with t0: 4.489 and last moment: 4.621
truncated: False, terminated: False
Counter overflow with t0: 4.654 and last moment: 4.786
truncated: False, terminated: True


(
    array([0.4509804 , 0.        , 0.        , 0.4509804 , 0.        ,
       0.        , 0.45490196, 0.        , 0.        , 0.45490196,
       0.        , 0.        , 0.42745098, 0.        , 0.        ,
       0.3647059 , 0.        , 0.        , 0.2       , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00784314, 0.00784314, 0.00784314, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.40784314, 0.        ,
       0.        , 0.46666667, 0.        , 0.        , 0.46666667,
       0.        , 0.        , 0.4627451 , 0.        , 0.        ,
       0.4509804 , 0.        , 0.        , 0.4509804 , 0.        ,
       0.        , 0.45490196, 0.        , 0.        , 0.45490196,
       0.        , 0.        , 0.42745098, 0.        , 0.        ,
       0.3647059 , 0.        , 0.        , 0.2       , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00784314, 0.00784314, 0.00784314, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.40784314, 0.        ,
       0.        , 0.46666667, 0.        , 0.        , 0.46666667,
       0.        , 0.        , 0.4627451 , 0.        , 0.        ,
       0.4509804 , 0.        , 0.        , 0.4509804 , 0.        ,
       0.        , 0.45490196, 0.        , 0.        , 0.45490196,
       0.        , 0.        , 0.42745098, 0.        , 0.        ,
       0.3647059 , 0.        , 0.        , 0.2       , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00784314, 0.00784314, 0.00784314, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.40784314, 0.        ,
       0.        , 0.46666667, 0.        , 0.        , 0.46666667,
       0.        , 0.        , 0.4627451 , 0.        , 0.        ],
      dtype=float32),
    -1.0,
    False,
    True,
    {}
)

# Robot Pursuit

In [2]:
system = ThreeWheelPushingObject()
sim_rob = Robot3PiRobotPursuit(system, 
                                state_init=np.zeros([2*50*3]),
                                time_final=30)

In [3]:
from src.objective import RobotPursuitRunningObjective


objective_function = RobotPursuitRunningObjective()
rp = RobotPursuit(sim_rob, running_objective=objective_function)

simulator: <src.simulator_gz.Robot3PiRobotPursuit object at 0x7f76addeb700>
Subscribed to Camera!
Subscribed to dynamic_pose/info! for catching robot
Subscribed to dynamic_pose/info! for running robot
Waiting for simulator... 0.32 sec
Scene received!
Model in scene arena_ground_plane
Model in scene ground_plane
Model in scene red_runner_robot
Model in scene green_runner_robot
Model in scene blue_runner_robot
Model in scene yellow_runner_robot
Model in scene catcher_robot

Observation and position received!


In [8]:
rp.switch_task(3)
observation, _ = rp.reset()

rp.task_list

receive self.task_id: yellow_cylinder
Stopping green_runner_robot!
parked runner green_runner_robot at [-30, -2, -0.25]
Stopping yellow_runner_robot!


['red_cube', 'green_capsule', 'blue_sphere', 'yellow_cylinder']

In [9]:
import time

observation, _ = rp.reset()

for _ in range(100):
    observation, reward, truncated, terminated, info = rp.step(np.array([1, 0]))
    print(f"truncated: {truncated}, terminated: {terminated}")
    if np.logical_or(truncated, terminated):
        break
    
    time.sleep(0.2) 

rp.step(np.array([0, 0]))

Stopping yellow_runner_robot!
[Objective Function] COND: Normal, REWARD: 0.5
truncated: False, terminated: False
[Objective Function] COND: Normal, REWARD: 0.3285714285714285
Counter overflow with t0: 9.901 and last moment: 10.0
truncated: False, terminated: False
[Objective Function] COND: Normal, REWARD: 0.30000000000000004
Counter overflow with t0: 10.0 and last moment: 10.132
truncated: False, terminated: False
[Objective Function] COND: Normal, REWARD: 0.21538461538461529
Counter overflow with t0: 10.132 and last moment: 10.264
truncated: False, terminated: False
[Objective Function] COND: Normal, REWARD: 0.19999999999999996
truncated: False, terminated: False
[Objective Function] COND: Normal, REWARD: 0.15000000000000002
Counter overflow with t0: 10.429 and last moment: 10.561
truncated: False, terminated: False
COND: LOST
Counter overflow with t0: 10.561 and last moment: 10.693
truncated: True, terminated: False
COND: LOST
Counter overflow with t0: 10.693 and last moment: 10.825

(
    array([0.85490197, 0.85490197, 0.85490197, ..., 0.2901961 , 0.2901961 ,
       0.2901961 ], dtype=float32),
    -1.0,
    True,
    False,
    {}
)